In [1]:
from quantum_badger import *
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats=['svg'] 

import pandas as pd

colors = ['#8ECAE6', '#219EBC', '#023047', '#FFCB47', '#FFB703', '#FB8500', '#BB0A21']

from numba import njit


## Initialize the Gaussian Boson Sampling Emulator

In [2]:
path = return_path(filename='demo.ipynb')

In [ ]:
# Set the GBS device parameters

random.seed(42)

# Number of modes 
m = 8
# Number of input squeezed states
n = round(m/2) 
# Squeezing parameter of the input squeezed vacuum states
r = 1.6

# Number of beam splitters 
n_BS = m**2

r_, phi_ = input_state(r, m, n) 
A = set_input(r_, phi_, path)
U = get_random_interferometer(m, n_BS, path)

# or 
# U = import_interferometer(path, '/matrix_U.dat') 

M = set_device_parameters(r, A, U, path)
# or 
# M, m, n, r, n_cutoff, n_mc, batch_size = import_input(path, "/GBS_matrix.dat")

# Tests
#check_set_parameters(U,M)

In [7]:
M, m, n, r, n_cutoff, n_mc, batch_size = import_input(path, "/GBS_matrix.dat")

Data were imported from /Users/anastasiacertkova/Desktop/Coding/GBS_matrix.dat


In [ ]:
error = 0.05

ind, phi, psi, eta = import_parameters_interferometer(path, '/parameters_of_interferometer.dat')
U_appr = interferometer_approx(n_BS, ind, phi, psi, eta, error,  m)

print("Frobenius distance between U and U_appr:", round(frobenius_distance(U, U_appr),2) )

In [ ]:
batch_size = 10 
samples = uniform_sampling_tr(batch_size,n,m)
samples

In [ ]:
export_samples(samples, path, "/samples.dat")

In [3]:
samples = import_samples(path, "/samples.dat")
samples

[[1, 1, 0, 1, 0, 1, 0, 0],
 [1, 0, 1, 0, 0, 0, 1, 1],
 [0, 0, 1, 0, 1, 1, 0, 1],
 [0, 0, 1, 0, 1, 1, 1, 0],
 [1, 1, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 1, 1, 1],
 [1, 0, 0, 0, 1, 0, 1, 1],
 [0, 1, 1, 0, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 1]]

## Run the Gaussian Boson Sampling Emulator

In [ ]:
%%bash

stat

In [ ]:
# cmd = "python3 " path + r"GBS.py" 
# call(cmd.split(" "))

## Compute probabilities exactly

In [ ]:
# samples, probabilities = import_gbs_samples(path)

In [8]:
probabilities_exact = []

for s in samples:
    
    probabilities_exact.append(prob_exact(s, M))

In [9]:
for i in range(len(samples)):
    print(samples[i],convert_01_0123(samples[i]), probabilities_exact[i])

[1, 1, 0, 1, 0, 1, 0, 0] [0, 1, 3, 5] 0.001540221685623145
[1, 0, 1, 0, 0, 0, 1, 1] [0, 2, 6, 7] 0.00038379315780583313
[0, 0, 1, 0, 1, 1, 0, 1] [2, 4, 5, 7] 3.2229636431950325e-05
[0, 0, 1, 0, 1, 1, 1, 0] [2, 4, 5, 6] 8.449216163451008e-06
[1, 1, 1, 0, 0, 0, 0, 1] [0, 1, 2, 7] 0.002032919617881158
[1, 0, 0, 0, 1, 0, 1, 1] [0, 4, 6, 7] 0.0002346692656297943
[0, 0, 1, 0, 0, 1, 1, 1] [2, 5, 6, 7] 4.571010886750454e-06
[1, 0, 0, 0, 1, 0, 1, 1] [0, 4, 6, 7] 0.0002346692656297943
[0, 1, 1, 0, 1, 0, 1, 0] [1, 2, 4, 6] 0.0003818290608265405
[1, 0, 0, 1, 0, 1, 0, 1] [0, 3, 5, 7] 0.00011005467768718512


In [ ]:
# Obtain all possible samples for theshold detection
all_permutations = threshold_basis_set(m)
# Calculate probabilities for all possible samples 

probabilities_exact = []

for s in all_permutations:
    probabilities_exact.append(prob_exact(s, M))
    

# Sum all probabilities to obtain 1 
print("sum prob:", sum(probabilities_exact))


In [ ]:
basis_dictionary = {
        str(all_permutations[i]): [sum(all_permutations[i]), probabilities_exact[i]] 
        for i in range(len(all_permutations))
    }

In [ ]:
df_basis = (
    pd.DataFrame
    .from_dict(basis_dictionary, orient='index', columns=["n_clicks","probability_exact"])
)

df_basis.index.name = "sample"


In [ ]:
df_basis.info()
df_basis.head()

In [ ]:
# Find probability of a specific sample in DataFrame
sample = [0]*m
df_basis["probability_exact"].loc[str(sample)] 

In [ ]:
pmf_values = [sum(df_basis["probability_exact"][df_basis["n_clicks"] == n]) for n in range(m+1)]

plt.vlines(
    range(m+1), 
    0,
    pmf_values, 
    color = colors[0],
    linestyles='dashed'
)

plt.plot(
    range(m+1), 
    pmf_values,
    'o-',
    color = colors[1]
)

plt.yscale('log')
plt.xlabel("Number of clicks", fontsize=12)
plt.ylabel("Probability mass function", fontsize=12)
plt.title("PMF for Gaussian Boson Sampling", fontsize=17);
plt.show()

In [ ]:
# Alignment 
# Create column "samples_frequency"
# Values should contain the frequencies of imported samples 

#df_basis["probability_exact"].loc[str(samples[0])]

In [ ]:
# Test: comparison exact probabilities with sf for the small problem

In [ ]:
# Compute probabilities for sectors exactly

def prob_sectors_exact(M, sample=[1]*len(M), nu_max=10*len(M)):
    
    """
    Calculates the exact probabilities over sectors for a given sample 
    and Gaussian matrix.

    Args:
        M (numpy.ndarray): The Gaussian matrix for computation.
        sample (list, optional): The sample to calculate the probabilities for. Defaults to [1]*len(M).
        nu_max (int, optional): The maximum value for the parameter 'nu'. Defaults to 10*len(M).

    Returns:
        numpy.ndarray: The exact probabilities over sectors for the given sample and Gaussian matrix.

        
    """
 
    
    clicked_detectors = convert_01_0123(sample)    
    M_sub = red_mat(M, clicked_detectors)

    dnu = 2 * np.pi / nu_max
    m = len(M_sub)

    stat = np.zeros((m + 1, nu_max), dtype=np.complex128)
    sectors = np.zeros((m + 1, nu_max), dtype=np.float64)
                    
    for i in range(m+1):
        detect_event = [1 for j in range(i)] + [0]*(m-i) 
        permutations = list(permut(detect_event))

        if i == 0:
            for nu in range(nu_max):
                stat[i,nu] += 1

        else:
            for nu in range(nu_max):
                for s in permutations:
                    stat[i,nu] += Z_i(s, M_sub, nu=nu*dnu) 
                    
    for k in range(m):
        for h in range(k + 1, m + 1):
            for nu in range(nu_max):
                stat[h, nu] -= stat[k, nu] * number_of_comb(m - k, m - h)

    for n in range(m + 1):
        for j in range(nu_max):
            for k in range(nu_max):
                sectors[n, j] += (stat[n, k]*np.exp(-1j*j*k*dnu)/nu_max).real
                
    return sectors/Z(M)

In [ ]:
nu_max =  m*10

P_sectors = prob_sectors_exact(M,sample=[1]*m, nu_max=nu_max)

for nu in range(m,m*10,m):
    plt.plot(
        range(m+1),
        [P_sectors[j,nu] for j in range(m+1)],
        '--' ,
        label = 'k='+str(nu)
)
plt.yscale('log')
plt.legend(prop={'size':10}, loc='lower left')
plt.xlabel("Number of clicks", fontsize=12)
plt.ylabel("Probability mass function", fontsize=12)
plt.title("PMF for sectors", fontsize=17);
plt.show()

In [ ]:
n_clicked = m
plt.plot(
        range(nu_max),
        [P_sectors[n_clicked,nu] for nu in range(nu_max)],
        '-' 
)
plt.yscale('log')
#plt.legend(prop={'size':10}, loc='lower left')
plt.xlabel("Sectors", fontsize=12)
plt.ylabel("Probability mass function", fontsize=12)
plt.title(f"PMF of sectors for {n_clicked} clicked detectors", fontsize=17);
plt.show()

In [ ]:
# Comparison of calculation of the exact probabilities with and without sectors
nu_max = 10*m
list_det = [1]*3 +[0]*(m-3)


P_sectors =  prob_sectors_exact(M, sample=list_det,nu_max=nu_max)

P_ = 0
for nu in range(nu_max):
    P_ += P_sectors[sum(list_det),nu]
    
print(P_, df_basis["probability_exact"].loc[str(list_det)])

In [45]:
def count_samples(samples, samples_dictionary):
    batch_size = len(samples)
    n_unique = len(samples_dictionary.keys())
    n_counts = [0]*n_unique

    unique_samples = list(samples_dictionary.keys())

    for i in range(batch_size):
        if str(samples[i]) in unique_samples:
            index =  unique_samples.index(str(samples[i]))
            n_counts[index]+=1
        else:
            raise 'Incomplete list of unique samples in the dictionary'
    return n_counts



In [48]:
# Export samples and exact probabilities? 

samples_dictionary = {
        str(s): [sum(s), prob_exact(s, M)] for s in samples
    }

df_experiment = (
    pd.DataFrame
    .from_dict(samples_dictionary, orient='index', columns=["n_clicks","probability_exact"])
)

df_experiment.index.name = "sample"

df_experiment["n_counts"] = count_samples(samples, samples_dictionary)

df_experiment.info()
df_experiment.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, [1, 1, 0, 1, 0, 1, 0, 0] to [1, 0, 0, 1, 0, 1, 0, 1]
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   n_clicks           9 non-null      int64  
 1   probability_exact  9 non-null      float64
 2   n_counts           9 non-null      int64  
dtypes: float64(1), int64(2)
memory usage: 288.0+ bytes


,n_clicks,probability_exact,n_counts
sample,,,
"[1, 1, 0, 1, 0, 1, 0, 0]",4,0.001540,1
"[1, 0, 1, 0, 0, 0, 1, 1]",4,0.000384,1
"[0, 0, 1, 0, 1, 1, 0, 1]",4,0.000032,1
"[0, 0, 1, 0, 1, 1, 1, 0]",4,0.000008,1
"[1, 1, 1, 0, 0, 0, 0, 1]",4,0.002033,1
"[1, 0, 0, 0, 1, 0, 1, 1]",4,0.000235,2
"[0, 0, 1, 0, 0, 1, 1, 1]",4,0.000005,1
"[0, 1, 1, 0, 1, 0, 1, 0]",4,0.000382,1
"[1, 0, 0, 1, 0, 1, 0, 1]",4,0.000110,1


## Compute submatrices according to samples

In [ ]:
# Export submatrices for samples
submatriсes_export(M, samples, path)

## Compute minors 

In [ ]:
compute_minors(path)

## Compute moments

In [ ]:
id_ = 1
moments = MomentUtility(id_ = id_, n_moments = 4)

In [ ]:
moments.export_moments()
#m1_, m2_, m3_, m4_ = moments.get_moments(path)

## Get approximate probabilities

In [ ]:
cumulants = CumulantUtility(id_ = id_, n_moments = 4)

probability_approx_2,probability_approx_3, probability_approx_4 = cumulants.prob_approx()

In [11]:
def get_approx_probabilities(path):
    probs_approx_2 = []
    probs_approx_3 = []
    probs_approx_4 = []

    data_ids = np.genfromtxt(path + '/input/samples_ids.dat', dtype=str)

    ids = [int(i) for i in data_ids[:,0]]

    for i in ids:
        moments = MomentUtility(id_ = i)
        moments.export_moments()
        cumulants = CumulantUtility(id_ = i)
        probability_approx_2, probability_approx_3, probability_approx_4 = cumulants.prob_approx()
        probs_approx_2.append(probability_approx_2)
        probs_approx_3.append(probability_approx_3)
        probs_approx_4.append(probability_approx_4)

    
    return probs_approx_2, probs_approx_3, probs_approx_4


probs_approx_2, probs_approx_3, probs_approx_4 = get_approx_probabilities(path)

In [14]:
probs_approx_2

[0.016723598247845695,
 0.0074267316868304786,
 0.0013491009265548077,
 0.00042318346742149554,
 0.020621778572468236,
 0.004174295494375053,
 0.00017442892630754357,
 0.004174295494375053,
 0.004537927799696737,
 0.002310366359987994]

In [24]:
len(samples)

10

In [22]:
df_experiment["probability_approx_2"] = probs_approx_2
# df_experiment["probability_approx_3"] = pd.Series(probs_approx_3)
# df_experiment["probability_approx_4"] = pd.Series(probs_approx_4)
df_experiment    

ValueError: Length of values (10) does not match length of index (9)

In [ ]:
#def get_minors(id_):
    

In [ ]:
df_experiment

In [ ]:
# Data moments 

# moment_0 = (Z_v_0f[:]/Z_v_0[0]).real
# moment_1 = data_mom[:, 0]
# moment_2 = data_mom[:, 1]
# moment_3 = data_mom[:, 2]
# moment_4 = data_mom[:, 3]

# # Model moments

# import scipy.stats as sts
# import scipy.integrate as intgr
# import scipy.optimize as opt

# xfx = lambda x: x * moment_0 * gauss_fun(x, moment_1, moment_2)

# xfx(1)
# (mean_model, m_m_err) = intgr.quad(xfx, -np.inf, cutoff)
# x2fx = lambda x: ((x - mean_model) ** 2) * trunc_norm_pdf(x, mu, sigma, cutoff) 
# (var_model, v_m_err) = intgr.quad(x2fx, -np.inf, cutoff)


